In [1]:
import random
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from convnets.utilities import utils

In [12]:
# dimensions of our images.
img_width, img_height = 224, 224
train_data_dir = '/home/mksnkv/Documents/classification/documents_5class_clean/train'
validation_data_dir = '/home/mksnkv/Documents/classification/documents_5class_clean/validation'

weights_path = '/home/mksnkv/models/top_tuned/imagenetV2/weights_5class.h5'
weights_path_mcp = '/home/mksnkv/models/top_tuned/imagenetV2/weights_5class_mcp.h5'
structure_path = '/home/mksnkv/models/top_tuned/imagenetV2/structure_5class.json'
nb_train_samples = 1200
nb_validation_samples = 325
nb_evaluation_samples = 560
epochs = 20
batch_size = 5

In [19]:
def train_model():
        # build the MobileNetV2 network
    base_model = mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), alpha=1.0, include_top=False, weights='imagenet')

    head_model = base_model.output
#     print(base_model.output.shape)
    head_model = GlobalAveragePooling2D()(head_model)
    head_model = Dense(5, activation='softmax')(head_model)
    model = Model(inputs= base_model.input, outputs= head_model)

    for layer in base_model.layers[:142]:
        layer.trainable = False
    model.summary()
    
    opt = Adam(learning_rate=1e-6)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy', metrics=['accuracy'])
#     earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
#     reduce_lr_acc = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='max')
    mcp_save = ModelCheckpoint(weights_path_mcp, save_best_only=True, monitor='val_loss', mode='min')
    history = model.fit_generator(
            generator=train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
             callbacks = [mcp_save],
            validation_data=val_generator,
            validation_steps=nb_validation_samples // batch_size)
    return model, history

In [20]:
train_generator, val_generator = utils.init_generators(train_data_dir, 
                                                       validation_data_dir, 
                                                       img_width, 
                                                       img_height, 
                                                       batch_size,
                                                       batch_size)

Found 1200 images belonging to 5 classes.
Found 325 images belonging to 5 classes.


In [ ]:
model, history = train_model()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

  ...
    to  
  ['...']
Train for 240 steps, validate for 65 steps
Epoch 1/20
240/240 [==============================] - 61s 252ms/step - loss: 1.6260 - accuracy: 0.2542 - val_loss: 1.6486 - val_accuracy: 0.2123
Epoch 2/20
240/240 [==============================] - 59s 245ms/step - loss: 1.4317 - accuracy: 0.3958 - val_loss: 1.5887 - val_accuracy: 0.2554
Epoch 3/20
240/240 [==============================] - 58s 244ms/step - loss: 1.3022 - accuracy: 0.4900 - val_loss: 1.5427 - val_accuracy: 0.2646
Epoch 4/20
240/240 [==============================] - 59s 244ms/step - loss: 1.1999 - accuracy: 0.5433 - val_loss: 1.5023 - val_accuracy: 0.2738
Epoch 5/20
240/240 [==============================] - 59s 244ms/step - loss: 1.0864 - accuracy: 0.6350 - val_loss: 1.4722 - val_accuracy: 0.2954
Epoch 6/20
240/240 [==============================] - 59s 245ms/step - loss: 0.9855 - accuracy: 0.6883 - val_loss: 1.4416 - val_accuracy: 0.3108
Epoch 7/20
240/240 [==============================] - 59s 244m

In [29]:
utils.plot(history)
model.save('/home/mksnkv/models/top_tuned/imagenetV2/whole_model_passport.h5')
model.save_weights(weights_path)
utils.save_model_structure_to_json(structure_path, model)

In [70]:
def train_model_with_3_prev_layers(model):
#     base_model = mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), alpha=1.0, include_top=False, weights='imagenet')


#     global_average_layer = GlobalAveragePooling2D()
#     prediction_layer = Dense(1, activation='sigmoid')
#     model = Sequential([
#           base_model,
#           global_average_layer,
#           prediction_layer
#     ])
# #     model = load_model_structure_from_json(structure_path)
#     freeze(model)
#     model.load_weights(weights_path)
#     unfreeze(model)
#     for layer in base_model.layers[:144]:
#         layer.trainable = False
    utils.unfreeze(model)
    base_model = model.layers[0]
    for layer in base_model.layers[:144]:
         layer.trainable = False
    model.summary()
    
    opt = Adam(learning_rate=1e-8)
    model.compile(optimizer=opt,
                  loss='binary_crossentropy', metrics=['binary_accuracy'])
    
#     earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
#     reduce_lr_acc = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='max')
#     mcp_save = ModelCheckpoint(weights_path, save_best_only=True, monitor='val_loss', mode='min')
    history = model.fit_generator(
            generator=train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=15,
#             callbacks = [reduce_lr_acc],
            validation_data=val_generator,
            validation_steps=nb_validation_samples // batch_size)
    return model, history

In [ ]:
train_generator.reset()
val_generator.reset()
model_tuned, history_tuned = train_model_with_3_prev_layers(model)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 887,361
Non-trainable params: 1,371,904
_________________________________________________________________
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 14 steps
Epoch 1/15
50/50 [==============================] - 23s 465ms/step - loss: 0.0753 - binary_accuracy: 0.9800 - val_loss: 0.2879 - val_binary_accuracy: 0.8857
Epoch 2/15
17/50 [=========>....................] - ETA: 13s - loss: 0.0872 - binary_accuracy: 0.9750

KeyboardInterrupt: 